In [1]:
# Setiment 140

In [1]:
import pandas as pd
import numpy as np

import io
import os
import re
import shutil
import string
import tensorflow as tf
import zipfile

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

###  a)

In [77]:
zf = zipfile.ZipFile('trainingandtestdata.zip')
headlist = ['Polarity','Tweet_id', 'Tweet_Date', 'Query', 'User', 'Tweet_text']
train = pd.read_csv(zf.open('trainingandtestdata/training.1600000.processed.noemoticon.csv'), encoding='ISO-8859-1', names = headlist)

In [78]:
train

,Polarity,Tweet_id,Tweet_Date,Query,User,Tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [79]:
test = pd.read_csv("trainingandtestdata/testdata.manual.2009.06.14.csv", encoding='ISO-8859-1', names = headlist)

In [80]:
test

,Polarity,Tweet_id,Tweet_Date,Query,User,Tweet_text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


###  b)

### What are Stopwords?

for example: Consider this text string – “There is a pen on the table”. Now, the words “is”, “a”, “on”, and  “the” add no meaning to the statement while parsing it. Whereas words like “there”, “book”, and “table” are the keywords and tell us what the statement is all about.

 ### Why do we nedd to remove stopwords?

Removing stopwords is not a hard and fast rule in NLP. It depends upon the task that we are working on. For tasks like text classification, where the text is to be classified into different categories, stopwords are removed or excluded from the given text so that more focus can be given to those words which define the meaning of the text.
However, in tasks like machine translation and text summarization, removing stopwords is not advisable.

Here are a few key benefits of removing stopwords:

On removing stopwords, dataset size decreases and the time to train the model also decreases
Removing stopwords can potentially help improve the performance as there are fewer and only meaningful tokens left. Thus, it could increase classification accuracy
Even search engines like Google remove stopwords for fast and relevant retrieval of data from the database

### When should we remove stopwords?

### remove stopwords

##### Text Classification: 
    Spam Filtering
    Language Classification
    
#### Genre Classification: 
    Caption Generation
    Auto-Tag Generation

### Avoid stopword removal

Machine Translation

Language Modeling

Text Summarization

Question-Answering problems

### remove stopwords for traindata

In [7]:
# list of stopwords in NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:
from gensim.parsing.preprocessing import remove_stopwords

/home/anna/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
result_train = []
for i in train.iloc[:,-1]:
    result_train.append(remove_stopwords(i))
result_train

["@switchfoot http://twitpic.com/2y1zl - Awww, that's bummer. You shoulda got David Carr Third Day it. ;D",
 "upset can't update Facebook texting it... result School today also. Blah!",
 '@Kenichan I dived times ball. Managed save 50% The rest bounds',
 'body feels itchy like',
 "@nationwideclass no, it's behaving all. i'm mad. here? I can't there.",
 '@Kwesidei crew',
 'Need hug',
 "@LOLTrish hey long time see! Yes.. Rains bit ,only bit LOL , I'm fine thanks , how's ?",
 "@Tatiana_K nope didn't",
 '@twittera que muera ?',
 "spring break plain city... it's snowing",
 'I re-pierced ears',
 "@caregiving I couldn't bear watch it. And I thought UA loss embarrassing . . . . .",
 '@octolinz16 It counts, idk I either. talk anymore',
 "@smarrison would've first, didn't gun. though, zac snyder's doucheclown.",
 '@iamjazzyfizzle I wish I got watch you!! I miss @iamlilnicki premiere?!',
 "Hollis' death scene hurt severely watch film wry directors cut now?",
 'file taxes',
 '@LettyA ahh ive wanted

In [10]:
train.iloc[0,-1]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [11]:
result_train[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's bummer. You shoulda got David Carr Third Day it. ;D"

### remove punctuations for traindata

In [12]:
import re

In [13]:
fin_result_train = []
for i in result_train:
    fin_result_train.append(re.sub(r'[^\w\s]', '', i))
fin_result_train

['switchfoot httptwitpiccom2y1zl  Awww thats bummer You shoulda got David Carr Third Day it D',
 'upset cant update Facebook texting it result School today also Blah',
 'Kenichan I dived times ball Managed save 50 The rest bounds',
 'body feels itchy like',
 'nationwideclass no its behaving all im mad here I cant there',
 'Kwesidei crew',
 'Need hug',
 'LOLTrish hey long time see Yes Rains bit only bit LOL  Im fine thanks  hows ',
 'Tatiana_K nope didnt',
 'twittera que muera ',
 'spring break plain city its snowing',
 'I repierced ears',
 'caregiving I couldnt bear watch it And I thought UA loss embarrassing     ',
 'octolinz16 It counts idk I either talk anymore',
 'smarrison wouldve first didnt gun though zac snyders doucheclown',
 'iamjazzyfizzle I wish I got watch you I miss iamlilnicki premiere',
 'Hollis death scene hurt severely watch film wry directors cut now',
 'file taxes',
 'LettyA ahh ive wanted rent love soundtrack',
 'FakerPattyPattz Oh dear Were drinking forgotten tabl

In [14]:
fin_result_train[0]

'switchfoot httptwitpiccom2y1zl  Awww thats bummer You shoulda got David Carr Third Day it D'

In [15]:
# Problem: if we delete all Punctuations, e.g ;D means smile. but after delete it only mean a letter D. It is very different.

In [16]:
train['Re_P_S'] = fin_result_train
train

,Tweet_id,Tweet_Date,Query,User,Tweet_text,Re_P_S
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccom2y1zl Awww thats bum...
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant update Facebook texting it result S...
0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,Kenichan I dived times ball Managed save 50 Th...
0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,body feels itchy like
0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no its behaving all im mad her...
...,...,...,...,...,...,...
4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,Just woke up Having school best feeling
4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,TheWDBcom Very cool hear old Walt interviews ...
4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Are ready MoJo Makeover Ask details
4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Happy 38th Birthday boo alll time Tupac Amaru ...


### remove stopwords for testdata

In [17]:
result_test = []
for i in test.iloc[:,-1]:
    result_test.append(remove_stopwords(i))
result_test

['@stellargirl I loooooooovvvvvveee Kindle2. Not DX cool, 2 fantastic right.',
 'Reading kindle2... Love it... Lee childs good read.',
 'Ok, assesment #kindle2 ...it fucking rocks!!!',
 "@kenburbary You'll love Kindle2. I've months looked back. The new big huge! No need remorse! :)",
 "@mikefish Fair enough. But Kindle2 I think it's perfect :)",
 "@richardebaker no. big. I'm happy Kindle2.",
 'Fuck economy. I hate aig non loan given asses.',
 'Jquery new best friend.',
 'Loves twitter',
 'love Obama? makes jokes himself.',
 "Check video -- President Obama White House Correspondents' Dinner http://bit.ly/IMXUM",
 "@Karoli I firmly believe Obama/Pelosi ZERO desire civil. It's charade slogan, want destroy conservatism",
 'House Correspondents dinner night whoopi, barbara &amp; sherri went, Obama got standing ovation',
 'Watchin Espn..Jus seen new Nike Commerical Puppet Lebron..sh*t hilarious...LMAO!!!',
 'dear nike, stop flywire. shit waste science. ugly. love, @vincentx24x',
 "#lebron be

In [18]:
test.iloc[0,-1]

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [19]:
result_test[0]

'@stellargirl I loooooooovvvvvveee Kindle2. Not DX cool, 2 fantastic right.'

### remove punctuations for testdata

In [20]:
fin_result_test = []
for i in result_test:
    fin_result_test.append(re.sub(r'[^\w\s]', '', i))
fin_result_test

['stellargirl I loooooooovvvvvveee Kindle2 Not DX cool 2 fantastic right',
 'Reading kindle2 Love it Lee childs good read',
 'Ok assesment kindle2 it fucking rocks',
 'kenburbary Youll love Kindle2 Ive months looked back The new big huge No need remorse ',
 'mikefish Fair enough But Kindle2 I think its perfect ',
 'richardebaker no big Im happy Kindle2',
 'Fuck economy I hate aig non loan given asses',
 'Jquery new best friend',
 'Loves twitter',
 'love Obama makes jokes himself',
 'Check video  President Obama White House Correspondents Dinner httpbitlyIMXUM',
 'Karoli I firmly believe ObamaPelosi ZERO desire civil Its charade slogan want destroy conservatism',
 'House Correspondents dinner night whoopi barbara amp sherri went Obama got standing ovation',
 'Watchin EspnJus seen new Nike Commerical Puppet Lebronsht hilariousLMAO',
 'dear nike stop flywire shit waste science ugly love vincentx24x',
 'lebron best athlete generation time basketball related I dont want intersport debates _

In [21]:
fin_result_test[0]

'stellargirl I loooooooovvvvvveee Kindle2 Not DX cool 2 fantastic right'

###  c)

 #### What is lemmatization and what are the common lemmatization methods?

##### The process of lemmatization in natural language processing involves working with words according to their root lexical components. It is used in natural language processing and natural language understanding in computer programming and artificial intelligence. In simpler forms, a method that switches any kind of a word to its base root mode is called Lemmatization. 
For Example:
“Studies” => “Study”

Methode: Wordnet Lemmatizer with NLTK, TextBlob Lemmatizer,TextBlob Lemmatizer with appropriate POS tag....

In [22]:
from textblob import TextBlob, Word
# lemmatize a word
word = 'stripes'
w = Word(word)
w.lemmatize()

'stripe'

In [23]:
# Define function to lemmatize each word with its POS tag
def lemmatize_with_postag(sentence_list):
    r = []
    for sentence in sentence_list:
        sent = TextBlob(sentence)
        tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        r.append(" ".join(lemmatized_list))
    return r

In [24]:
# Lemmatize for train
lemma_train = lemmatize_with_postag(fin_result_train)

In [25]:
# Lemmatize for train
lemma_test = lemmatize_with_postag(fin_result_test)

In [26]:
fin_result_train[3]

'body feels itchy like'

In [27]:
fin_result_test[3]

'kenburbary Youll love Kindle2 Ive months looked back The new big huge No need remorse '

In [28]:
train['After_Lemmati'] = lemma_train

In [29]:
test['After_Lemmati'] = lemma_test

### d) What are word embeddings and what are the common methods used in creating word embedding?

#### Word embeddings are a way to represent words and whole sentences in a numerical manner. We know that computers understand the language of numbers, so we try to encode words in a sentence to numbers such that the computer can read it and process it. 

Mothode: Neural language Model, Word2Vec, Continuous bag-of-words model, Word embedding model using Pre-trained models, Continuous skip-gram model